In [1]:
# paramter cell do not remove!!
# nb_parm='datalake|raw/pdf|Birddiversityanddistribution|pdf||300|150'
# nb_parm='datalake|raw/text-csv|PFW_spp_translation_table_May2024|csv||300|150'
nb_parm='llmnok'
question = 'how many bird species are in migratory?'
embed_model = "mxbai-embed-large" 
gen_model = "deepseek-r1:7b"
# collection = "Bridknowledge"

In [2]:
import sys
import os, io

sys.path.append("/home/jovyan/notebooks")

## Do the task after this

### Register model

In [3]:
import subprocess
from pathlib import Path
import os
import shutil

# Step 1: Copy source folder to a clean new target location
source_dir = Path("/tmp/ADK/adk_short_bot")
target_dir = Path("/tmp/ADK/packaged_agent/adk_short_bot")

#remove .env file
def ignore_dotfiles(dir, files):
    return [f for f in files if f.startswith('.')]

# Remove target_dir if it already exists
if target_dir.exists():
    shutil.rmtree(target_dir)
shutil.copytree(source_dir, target_dir, ignore=ignore_dotfiles)



# Step 2: Restructure into proper Python package
flat_dir = target_dir
pkg_name = "adk_short_bot"
pkg_dir = flat_dir / pkg_name
dist_dir = flat_dir / "dist"

# Step 1: Move Python files into adk_short_bot/ subfolder
if not pkg_dir.exists():
    pkg_dir.mkdir()
    for file in flat_dir.glob("*.py"):
        if file.name != "setup.py":
            shutil.move(str(file), str(pkg_dir / file.name))

# Step 2: Move folders (like 'tools') into the package dir
for sub in flat_dir.iterdir():
    if sub.is_dir() and sub.name != pkg_name and sub.name != "dist":
        shutil.move(str(sub), str(pkg_dir / sub.name))

# Step 3: Ensure __init__.py exists
(pkg_dir / "__init__.py").touch(exist_ok=True)

# Step 4: Write or overwrite setup.py
setup_code = f"""
from setuptools import setup, find_packages

setup(
    name="{pkg_name}",
    version="0.1.0",
    description="A short bot agent using Vertex Reasoning Engines",
    packages=find_packages(),
    install_requires=[
        "google-cloud-aiplatform[adk,agent_engines]",
    ],
    python_requires=">=3.8",
)
"""
with open(flat_dir / "setup.py", "w") as f:
    f.write(setup_code)

# Step 5: Build and install the package
subprocess.run(["python", "-m", "build", "--outdir", str(dist_dir)], cwd=flat_dir, check=True)
subprocess.run(["pip", "install", str(list(dist_dir.glob("*.whl"))[0])], check=True)


* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - setuptools >= 40.8.0
* Getting build dependencies for sdist...
running egg_info
creating adk_short_bot.egg-info
writing adk_short_bot.egg-info/PKG-INFO
writing dependency_links to adk_short_bot.egg-info/dependency_links.txt
writing requirements to adk_short_bot.egg-info/requires.txt
writing top-level names to adk_short_bot.egg-info/top_level.txt
writing manifest file 'adk_short_bot.egg-info/SOURCES.txt'
reading manifest file 'adk_short_bot.egg-info/SOURCES.txt'
writing manifest file 'adk_short_bot.egg-info/SOURCES.txt'
* Building sdist...
running sdist
running egg_info
writing adk_short_bot.egg-info/PKG-INFO
writing dependency_links to adk_short_bot.egg-info/dependency_links.txt
writing requirements to adk_short_bot.egg-info/requires.txt
writing top-level names to adk_short_bot.egg-info/top_level.txt
reading manifest file 'adk_short_bot.egg-info/SOURCES.txt'
writing manifest file 'adk_short_

copying adk_short_bot/prompt.py -> adk_short_bot-0.1.0/adk_short_bot
copying adk_short_bot.egg-info/PKG-INFO -> adk_short_bot-0.1.0/adk_short_bot.egg-info
copying adk_short_bot.egg-info/SOURCES.txt -> adk_short_bot-0.1.0/adk_short_bot.egg-info
copying adk_short_bot.egg-info/dependency_links.txt -> adk_short_bot-0.1.0/adk_short_bot.egg-info
copying adk_short_bot.egg-info/requires.txt -> adk_short_bot-0.1.0/adk_short_bot.egg-info
copying adk_short_bot.egg-info/top_level.txt -> adk_short_bot-0.1.0/adk_short_bot.egg-info
copying adk_short_bot/tools/__init__.py -> adk_short_bot-0.1.0/adk_short_bot/tools
copying adk_short_bot/tools/character_counter.py -> adk_short_bot-0.1.0/adk_short_bot/tools
copying adk_short_bot.egg-info/SOURCES.txt -> adk_short_bot-0.1.0/adk_short_bot.egg-info
Writing adk_short_bot-0.1.0/setup.cfg
Creating tar archive
removing 'adk_short_bot-0.1.0' (and everything under it)
* Building wheel from sdist
* Creating isolated environment: venv+pip...
* Installing packages in

adk-short-bot is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


CompletedProcess(args=['pip', 'install', '/tmp/ADK/packaged_agent/adk_short_bot/dist/adk_short_bot-0.1.0-py3-none-any.whl'], returncode=0)

In [5]:
# Upload to MinIO (optional)
from Framework import base_tools

for fpath in dist_dir.glob("*"):
    key = f"adk_short_bot/{fpath.name}"    
    base_tools.upload_filepath_to_datalake(str(fpath), 'datalake', f'Agent/{key}')
    
    print(f"☁️ Uploaded {fpath.name} to s3://datalake/Agent/{key}")

UPLOAD FILE TO S3://datalake/Agent/adk_short_bot/adk_short_bot-0.1.0-py3-none-any.whl SUCCESSFULY!
☁️ Uploaded adk_short_bot-0.1.0-py3-none-any.whl to s3://datalake/Agent/adk_short_bot/adk_short_bot-0.1.0-py3-none-any.whl
UPLOAD FILE TO S3://datalake/Agent/adk_short_bot/adk_short_bot-0.1.0.tar.gz SUCCESSFULY!
☁️ Uploaded adk_short_bot-0.1.0.tar.gz to s3://datalake/Agent/adk_short_bot/adk_short_bot-0.1.0.tar.gz


In [6]:
import os
import boto3
from pathlib import Path
import subprocess
from botocore.config import Config

# === Configuration ===
minio_endpoint = os.getenv("MINIO_ENDPOINT", "http://localhost:9001/browser")
minio_access_key = os.getenv("MINIO_ACCESS_KEY", "minioadmin")
minio_secret_key = os.getenv("MINIO_SECRET_KEY", "minioadmin")
minio_bucket = "datalake"

minio_key = "Agent/adk_short_bot/adk_short_bot-0.1.0-py3-none-any.whl"  # or .tar.gz if needed
# minio_key = "Agent/adk_short_bot/adk_short_bot-0.1.0.tar.gz"
local_pkg_path = Path("/tmp/registered_agent") / Path(minio_key).name


# === Ensure download directory exists ===
local_pkg_path.parent.mkdir(parents=True, exist_ok=True)

# === Download from MinIO ===
s3 = boto3.client(
    's3',
    endpoint_url='http://minio:9000',  # Replace with your MinIO server URL
    aws_access_key_id='minioadmin',    # Use MinIO's access key
    aws_secret_access_key='minioadmin',# Use MinIO's secret key
    config=Config(signature_version='s3v4')
)

print(f"⬇️  Downloading {minio_key} from MinIO...")
s3.download_file(minio_bucket, minio_key, str(local_pkg_path))
print(f"✅ Downloaded to: {local_pkg_path}")

# === Uninstall package if found ===
package_name = "adk_short_bot"
print(f"⛔ Uninstalling any existing versions of {package_name}...")
subprocess.run(["pip", "uninstall", package_name, "-y"], check=False)

# === Install package ===
print(f"📦 Installing {local_pkg_path.name}...")
subprocess.run(["pip", "install", str(local_pkg_path)], check=True)
print("✅ Package installed.")


⬇️  Downloading Agent/adk_short_bot/adk_short_bot-0.1.0-py3-none-any.whl from MinIO...
✅ Downloaded to: /tmp/registered_agent/adk_short_bot-0.1.0-py3-none-any.whl
⛔ Uninstalling any existing versions of adk_short_bot...
Found existing installation: adk_short_bot 0.1.0
Uninstalling adk_short_bot-0.1.0:
  Successfully uninstalled adk_short_bot-0.1.0
📦 Installing adk_short_bot-0.1.0-py3-none-any.whl...
Processing /tmp/registered_agent/adk_short_bot-0.1.0-py3-none-any.whl


✅ Package installed.
